In [1]:
# ------- SERVER EXTENSIONS ---------
lib =  r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities/droplet_dataset'
lib2 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities'
lib3 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/data_analysis'
lib4 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy'
lib5 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/scripts'
import sys
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)
from utilities.general_helpers import *
# ------- SERVER EXTENSIONS ---------
import numpy as np
import pandas as pd
import scipy
import sklearn
from sklearn.manifold import TSNE
import pickle
# from Bio.Cluster import kcluster
import os
import numpy as np
import yaml
from os.path import join
import os
import pandas
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sys
import seaborn as sns
# import statsmodels as sm
import scipy.stats as stats
from scipy.stats import rankdata
from Bio.Cluster import kcluster
from sklearn.manifold import TSNE
import pickle
import numpy as np
from utilities.general_helpers import flatten_list
from utilities.ML_environment import find_marker_genes_in_cluster
# import pyclustering
from shutil import copyfile
import matplotlib.pyplot as plt

# Finding a threshold for variance so that we will have 4k~ genes.

##### Loads pickle file of NORMALIZED cohort

In [2]:
cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/cohort_normalized_24.5.21.pkl'
normalized_cohort = pickle.load(open(cohort_path, 'rb'))

##### Finds a threshold for all cohort

In [21]:
variances = np.var(normalized_cohort.counts, axis=0)
variances_sorted = np.flip(variances[np.argsort(variances)])

In [30]:
num_requested_genes = 4000
print('The 20 variances around the 4Kth gene:')
print(variances_sorted[num_requested_genes-10: num_requested_genes+10], end='\n\n')
print(f'To get 4K~ genes, you need to take a number between {variances_sorted[num_requested_genes-1: num_requested_genes+1]}')

The 20 variances around the 4Kth gene:
[0.3180053  0.31789164 0.31782625 0.31779738 0.31765869 0.31751089
 0.31746045 0.31741887 0.31738702 0.31736216 0.31734278 0.31726301
 0.31718303 0.31710028 0.31701251 0.31688825 0.31673426 0.31666172
 0.31660014 0.31650064]

To get 4K~ genes, you need to take a number between [0.31736216 0.31734278]


##### Finds a threshold for immune cells

In [3]:
immune_cells = normalized_cohort.filter_cells_by_property('is_immune', True)

In [4]:
variances = np.var(immune_cells.counts, axis=0)
variances_sorted = np.flip(variances[np.argsort(variances)])

In [5]:
num_requested_genes = 4000
print('The 20 variances around the 4Kth gene:')
print(variances_sorted[num_requested_genes-10: num_requested_genes+10], end='\n\n')
print(f'To get 4K~ genes, you need to take a number between {variances_sorted[num_requested_genes-1: num_requested_genes+1]}')

The 20 variances around the 4Kth gene:
[0.31913986 0.31913009 0.31912917 0.3190953  0.31892485 0.31877284
 0.31874335 0.3187426  0.31873801 0.31870492 0.31857896 0.31847667
 0.31841385 0.31832409 0.31831431 0.31823881 0.31820728 0.31818568
 0.31814925 0.31807193]

To get 4K~ genes, you need to take a number between [0.31870492 0.31857896]


##### Finds a threshold for tumor cells

In [28]:
tumor_cells = normalized_cohort.filter_cells_by_property('is_cancer', True)


In [33]:
variances = np.var(tumor_cells.counts, axis=0)
variances_sorted = np.flip(variances[np.argsort(variances)])

In [34]:
num_requested_genes = 4000
print('The 20 variances around the 4Kth gene:')
print(variances_sorted[num_requested_genes-10: num_requested_genes+10], end='\n\n')
print(f'To get 4K~ genes, you need to take a number between {variances_sorted[num_requested_genes-1: num_requested_genes+1]}')

The 20 variances around the 4Kth gene:
[0.27910708 0.27900916 0.27900727 0.27900416 0.27881262 0.27876171
 0.27874488 0.27858683 0.27837879 0.27834315 0.27812552 0.27787858
 0.27770549 0.27761339 0.27752057 0.27746356 0.27737847 0.27733027
 0.27725518 0.27721452]

To get 4K~ genes, you need to take a number between [0.27834315 0.27812552]


### Saves PKLs of immune, tumor, or all cells with after variance filter

Tumor:

In [71]:
out_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/tumor_cells_4k_genes.pkl'
variance = 0.2782 #[0.27834315 0.27812552]

tumor_filtered = tumor_cells.filter_genes_by_variance(variance)
pickle.dump((tumor_filtered), open(out_path, 'wb'), protocol=4)
print('Done')

Dataset was cleared from genes with variance of less than 0.2782
Done


Immune:

In [6]:
out_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/immune_cells_4k_genes.pkl'
variance = 0.3186  # [0.31870492 0.31857896]

immune_filtered = immune_cells.filter_genes_by_variance(variance)
pickle.dump((immune_filtered), open(out_path, 'wb'), protocol=4)
print('Done')

Dataset was cleared from genes with variance of less than 0.3186
Done


All cells (cohort):

In [ ]:
out_path = r''
variance = #[0.31736216 0.31734278]

cohort_filtered = normalized_cohort.filter_genes_by_variance(variance)
pickle.dump((cohort_filtered), open(out_path, 'wb'), protocol=4)
print('Done')

# Debug

In [39]:
tumor_cells.number_of_cells
sum(tumor_cells.cells_information.getattr('is_cancer'))

97225

In [44]:
ii = pickle.load(open(r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/immune_cells_4k_genes.pkl', 'rb'))

In [56]:
ii.counts[700:900].max()

10.936111592118117

In [77]:
sum(tumor_filtered.cells_information.getattr('is_cancer'))
tumor_filtered.number_of_cells

97225

In [62]:
cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/cohort_normalized_10.5.21.pkl'
normalized_cohort = pickle.load(open(cohort_path, 'rb'))

In [66]:
save_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/immune_cells_10.5.21.pkl'
immune_cells = normalized_cohort.filter_cells_by_property('is_immune', True)

In [70]:
immune_cells.number_of_genes
pickle.dump((immune_cells), open(save_path, 'wb'), protocol=4)

In [7]:
tt = pickle.load(open(out_path, 'rb'))
tt


In [9]:
tt.number_of_genes

4000